In [ ]:
import numpy as np
import keyboard
from IPython.display import clear_output
from src.constants import constants as cts
from src.model.MDP_2048 import MDP_2048

def get_action(last):
    curr = np.zeros(4, dtype=int)
    if keyboard.is_pressed("right"):
        curr[0] = 1
    if keyboard.is_pressed("left"):
        curr[1] = 1
    if keyboard.is_pressed("up"):
        curr[2] = 1
    if keyboard.is_pressed("down"):
        curr[3] = 1

    if curr.sum() != 1 or np.equal(last, curr).all():
        return None, curr

    if curr[0] == 1:
        return cts.MOVE_RIGHT, curr
    if curr[1] == 1:
        return cts.MOVE_LEFT, curr
    if curr[2] == 1:
        return cts.MOVE_UP, curr
    if curr[3] == 1:
        return cts.MOVE_DOWN, curr

last_key = np.zeros(4, dtype=int)
reward = 0
model = MDP_2048()
model.initialize_state()
print("SCORE:", reward)
print("\n\n\n")
model.print_state()

while not keyboard.is_pressed("esc"):

    action, last_key = get_action(last_key)

    if action is not None:
        model.transition_function(action)
        reward = model.reward_function()

        clear_output(wait=True)
        print("SCORE:", reward)
        print(f"\n{action}\n")
        model.print_state()

print("\nGAME ENDED, SCORE =", reward)

In [ ]:
def __rotate_before_compact(state, action):
    if action == cts.MOVE_RIGHT:
        return np.flip(state, axis=1)
    elif action == cts.MOVE_UP:
        return state.T
    elif action == cts.MOVE_DOWN:
        return np.flip(state.T, axis=1)
    else:
        return state

def __compact_tiles(state):
    def compact_row(row):
        length = len(row)
        row = row[np.where(row > 0)]
        for i in range(len(row) - 1):
            if row[i] == row[i + 1] & row[i] != 0:
                row[i] += 1
                row[i + 1] = 0
        row = row[np.where(row > 0)]
        row = np.pad(row, (0, length - len(row)), 'constant', constant_values=0)
        return row

    return np.apply_along_axis(compact_row, 1, state)

def __rotate_after_compact(state, action):
    if action == cts.MOVE_RIGHT:
        return np.flip(state, axis=1)
    elif action == cts.MOVE_UP:
        return state.T
    elif action == cts.MOVE_DOWN:
        return np.flip(state, axis=1).T
    else:
        return state


s = np.array([[1,2,1,0],[1,2,0,0],[3,0,0,0],[3,0,0,1]])
action = cts.MOVE_RIGHT

print(s, "init")
s = __rotate_before_compact(s, action)
print(s, "after first rotate")
s = __compact_tiles(s)
print(s, "after compact")
s = __rotate_after_compact(s, action)
print(s, "after second rotate")

In [ ]:
def compact_row(row):
    length = len(row)
    row = row[np.where(row > 0)]
    print(row)
    for i in range(len(row) - 1):
        if row[i] == row[i + 1] and row[i] != 0:
            row[i] += 1
            row[i + 1] = 0
    print(row)
    row = row[np.where(row > 0)]
    print(row)
    row = np.pad(row, (0, length - len(row)), 'constant', constant_values=0)
    print(row)
    return row

r = np.array([1, 3, 4, 1])
print(r)
r = compact_row(r)
print(r)